In [ ]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import time
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as scs
import numpy as np

### **2. Method 1: Using CoinMarketCap API**

**2.1 Get Your API Key & Configure API Access**

In [ ]:
API_KEY = 'e72c1278-e832-4f57-894a-bdfcd7ebfdca'
BASE_URL = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'

**Define parameters for the API request**

In [ ]:
headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': API_KEY,
}

parameters = {
    'start':'1',
    'limit':'1000',
    'convert':'USD'
}

### **2.3 Fetch Data from API**

In [ ]:
print("Attempting to fetch data from CoinMarketCap API...")
try:
    response = requests.get(BASE_URL, headers=headers, params=parameters)
    response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
    api_data_raw = json.loads(response.text)


    crypto_data_api = []
    if 'data' in api_data_raw:
        for coin in api_data_raw['data']:
            crypto_data_api.append({
                "Name": coin.get('name', 'N/A'),
                "Symbol": coin.get('symbol', 'N/A'),
                "Price (USD)": coin['quote']['USD'].get('price', 'N/A'),
                "Market Cap (USD)": coin['quote']['USD'].get('market_cap', 'N/A'),
                "Volume (24h, USD)": coin['quote']['USD'].get('volume_24h', 'N/A'),
                "Percent Change 1h": coin['quote']['USD'].get('percent_change_1h', 'N/A'),
                "Percent Change 24h": coin['quote']['USD'].get('percent_change_24h', 'N/A'),
                "Percent Change 7d": coin['quote']['USD'].get('percent_change_7d', 'N/A'),
                "Circulating Supply": coin.get('circulating_supply', 'N/A'),
                "Total Supply": coin.get('total_supply', 'N/A'),
                "Max Supply": coin.get('max_supply', 'N/A'),
                "Rank": coin.get('cmc_rank', 'N/A'),
                "Last Updated": coin['quote']['USD'].get('last_updated', 'N/A')
            })
        df_api = pd.DataFrame(crypto_data_api)
        print("\nSuccessfully fetched data from CoinMarketCap API:")
        print(df_api.head())
        print(f"\nTotal records fetched: {len(df_api)}")
    else:
        print("No 'data' field found in the API response. Check API key and request details.")
        print(api_data_raw) # Print raw response for debugging
        df_api = pd.DataFrame()

except requests.exceptions.RequestException as e:
    print(f"Error fetching data from CoinMarketCap API: {e}")
    print("Please ensure your API_KEY is correct and you have an active internet connection.")
    df_api = pd.DataFrame()
except json.JSONDecodeError:
    print("Error decoding JSON response from API. Response might not be valid JSON.")
    df_api = pd.DataFrame()
except KeyError as e:
    print(f"Missing expected key in API response: {e}. The API response structure might have changed or you're using a sandbox key with a live URL, or vice versa.")
    df_api = pd.DataFrame()
except Exception as e:
    print(f"An unexpected error occurred during API data fetching: {e}")
    df_api = pd.DataFrame()

### **3. Method 2: Web Scraping CoinMarketCap.com**

**3.1 Define Web Scraping Function**

In [ ]:
def scrape_coinmarketcap_website():
    url = "https://coinmarketcap.com/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    print(f"Attempting to scrape data from {url}...")
    try:
        response = requests.get(url, headers=headers, timeout=10) # Add a timeout
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page for web scraping: {e}")
        return pd.DataFrame()

    soup = BeautifulSoup(response.text, 'html.parser')

    data_scraped = []

    table = soup.find('table', class_='cmc-table') # Try to find the table by a common class
    if not table:
        print("Could not find the main cryptocurrency table. HTML structure might have changed.")
        rows = soup.select("tbody tr")
    else:
        rows = table.find('tbody').find_all('tr') # Get all table rows

    if not rows:
        print("No table rows found for cryptocurrencies. HTML structure might have changed or data not loaded.")
        return pd.DataFrame()

    print(f"Found {len(rows)} potential cryptocurrency rows. Parsing...")

    for i, row in enumerate(rows):
        # if i >= 50:
        #     break

        try:
            # Extract Name and Symbol
            name_elem = row.find('p', class_='coin-item-name')
            symbol_elem = row.find('p', class_='coin-item-symbol')
            name = name_elem.text.strip() if name_elem else 'N/A'
            symbol = symbol_elem.text.strip() if symbol_elem else 'N/A'

            price_elem = row.find('div', class_='sc-a0357053-0') # A common div wrapping prices

            if not price_elem: # Try another common class for price value
                price_elem = row.find('a', class_='cmc-link') # sometimes price is in an 'a' tag
            if price_elem:
                price_text = price_elem.find('span') # price value might be inside a span within the div/a
                price = price_text.text.strip() if price_text else price_elem.text.strip()
            else:
                price = 'N/A'


            market_cap_elem = row.find('td', {'data-cmc-sort-type': 'market_cap'})
            market_cap = market_cap_elem.find('div').text.strip() if market_cap_elem and market_cap_elem.find('div') else 'N/A'

            volume_elem = row.find('td', {'data-cmc-sort-type': 'volume_24h'})
            volume = volume_elem.find('div').text.strip() if volume_elem and volume_elem.find('div') else 'N/A'

            # Rank
            rank_elem = row.find('td', class_='cmc-table__cell--sticky-left') # Rank often has a sticky left cell
            rank = rank_elem.find('p').text.strip() if rank_elem and rank_elem.find('p') else 'N/A'


            data_scraped.append({
                "Rank": rank,
                "Name": name,
                "Symbol": symbol,
                "Price": price,
                "Market Cap": market_cap,
                "Volume (24h)": volume
            })
        except AttributeError:
            continue # Skip rows that don't have the expected elements
        except Exception as e:
            print(f"Error parsing row {i+1} during web scraping: {e}")
            continue

    df_scraped = pd.DataFrame(data_scraped)
    if not df_scraped.empty:
        print("\nSuccessfully scraped data from CoinMarketCap website (partial view):")
        print(df_scraped.head())
        print(f"\nTotal records scraped: {len(df_scraped)}")
    else:
        print("\nWeb scraping returned an empty DataFrame. This is common due to dynamic content or changing HTML structure.")
    return df_scraped

In [ ]:
scrape_coinmarketcap_website()

### **4. Integrating with Your Existing Files**

**4.1 Loading `coinmarket.json`**

In [ ]:
try:
    with open('coinmarket.json', 'r') as f:
        json_data = json.load(f)

    if 'fields' in json_data and 'values' in json_data:
        df_json = pd.DataFrame(json_data['values'], columns=json_data['fields'])
        print("\nData loaded from coinmarket.json:")
        print(df_json.head())
        print(f"\nTotal records from coinmarket.json: {len(df_json)}")
    else:
        print("\n'coinmarket.json' does not contain 'fields' and 'values' keys in the expected format.")
        df_json = pd.DataFrame()

except FileNotFoundError:
    print("\n'coinmarket.json' not found. Please ensure it's in the correct directory.")
    df_json = pd.DataFrame()
except json.JSONDecodeError:
    print("\nError decoding 'coinmarket.json'. It might not be a valid JSON file.")
    df_json = pd.DataFrame()
except Exception as e:
    print(f"\nAn error occurred while loading coinmarket.json: {e}")
    df_json = pd.DataFrame()

**4.2 Saving Data**

In [ ]:
if not df_api.empty:
    output_filename = '..../coinmarket.csv'
    df_api.to_csv(output_filename, index=False)
    print(f"\nAPI data saved to {output_filename}")